# Challenge 1: Binary Detection 1

The first step is to import relevant libraries.

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

The second step is to import data from the given file.

In [71]:
df = pd.DataFrame.from_csv("2challenge.csv")
df0 = df.loc[df['label'] == 0.0]
df1 = df.loc[df['label'] == 1.0]
dftest = df.loc[~((df['label'] == 0.0) | (df['label'] == 1.0))]
print(df0.shape)
print(df1.shape)
print(dftest.shape)

(5000, 9)
(5000, 9)
(5000, 9)


One can use the data in their original Pandas DataFrame format, or one can transform these objects into Numpy-arrays.

In [72]:
TrainingData0 = df0.as_matrix(columns=None)
TrainingData1 = df1.as_matrix(columns=None)
TestData = dftest.as_matrix(columns=['Y0', 'Y1','Y2','Y3','Y4','Y5','Y6','Y7'])

It may be helpful to visualize the data.

In [73]:
# Function to estimate missing data using normal distribution with mean and variance of given data in corresponding column
def Data_Estimator(nparray):
    blank = np.argwhere(np.isnan(nparray))
    lt_dim = len(nparray) - len(blank)
    local_temp = np.zeros(lt_dim)
    for i in range(len(blank)):
        nparray[blank[i]] = -1      # Marking blank space distinctly
    cnt = 0;
    for j in range(len(nparray)):
        if nparray[j] != -1:
            local_temp[cnt] = nparray[j]
            cnt = cnt + 1
    sigma = math.sqrt(np.var(local_temp))
    mu = np.mean(local_temp)
    for k in range(len(nparray)):
        if nparray[k] == -1:
            nparray[k] = np.random.normal(mu,sigma,1)
    return nparray;



After creating an algorithm and generating labels, one should update the original CSV file.

In [74]:
# Idea behind the algorithm is to select the test point and take the closest 25 training points to that point.
# The label of the most closest number of points decides the label for the test data point in question. I choose
# 25 points for the proximity test because the borderline estimation criteria is 13 closest points of a particular label, 
# which is ~52% of the total sample points, which is closer to the prioris given(50% each). I choose not to increase 
# the proximity points in order to sustain computation time. 

T_Array = np.append(TrainingData0, TrainingData1, axis=0)
T_Data = np.array(TestData)

TrainingData0[:,6] = Data_Estimator(TrainingData0[:,6])
TrainingData0[:,7] = Data_Estimator(TrainingData0[:,7])
TrainingData1[:,6] = Data_Estimator(TrainingData1[:,6])
TrainingData1[:,7] = Data_Estimator(TrainingData1[:,7])
TestData[:,6] = Data_Estimator(TestData[:,6])
TestData[:,7] = Data_Estimator(TestData[:,7])

Training_Array = np.append(TrainingData0, TrainingData1, axis=0);
new_c0 = 0;
new_c1 = 0;


num_data_points = TestData.shape[0];
num_training_points = Training_Array.shape[0];
estimation = np.zeros((num_data_points,1));

for i in range(num_data_points):
    y0 = TestData[i,0];
    y1 = TestData[i,1];
    y2 = TestData[i,2];
    y3 = TestData[i,3];
    y4 = TestData[i,4];
    y5 = TestData[i,5];
    y6 = TestData[i,6];
    y7 = TestData[i,7];
    Distance_Vector = np.zeros((num_training_points,2));
    count_0 = 0;
    count_1 = 0;
    for j in range(num_training_points):
        x0 = Training_Array[j,0];
        x1 = Training_Array[j,1];
        x2 = Training_Array[j,2];
        x3 = Training_Array[j,3];
        x4 = Training_Array[j,4];
        x5 = Training_Array[j,5];
        x6 = Training_Array[j,6];
        x7 = Training_Array[j,7];
        dist = math.sqrt((x0 - y0)**2 + (x1 - y1)**2 + + (x2 - y2)**2 + (x3 - y3)**2 + (x4 - y4)**2 + (x5 - y5)**2 + + (x6 - y6)**2 + (x7 - y7)**2);
        Distance_Vector[j,0] = dist;
        Distance_Vector[j,1] = Training_Array[j,8];
    A = Distance_Vector[:,0];
    min_idx = np.argpartition(A,25);
    for p in range(25):
        if Distance_Vector[min_idx[p],1] == 0:
            count_0 = count_0 + 1;
        else:
            count_1 = count_1 + 1;
    
       
    if count_1 > count_0:
        new_c1 = new_c1 + 1;
        estimation[i] = 1;
    else:
        new_c0 = new_c0 + 1;
        estimation[i] = 0;
        
Temp_Data_Array = np.append(T_Data,estimation,axis=1);
Array = np.append(T_Array,Temp_Data_Array,axis=0);
print ("Label0: , Label1: ", new_c0, new_c1);




('Label0: , Label1: ', 239, 4761)


In [75]:
dfupdate = pd.DataFrame({'Y0':Array[:,0],'Y1':Array[:,1],'Y2':Array[:,2],'Y3':Array[:,3],'Y4':Array[:,4],'Y5':Array[:,5],'Y6':Array[:,6],'Y7':Array[:,7],'label':Array[:,8]});
dfupdate.to_csv("2challenge.csv")